In [1]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'NLP-with-Disaster-Tweets-cleaning-data-V2'
device = 'cuda'

In [4]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [5]:
tokenize('$100')

['$', '100']

In [6]:
def stem(word):
    return stemmer.stem(word.lower())

In [7]:
stem('organic')

'organ'

In [8]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [9]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [10]:
data = pd.read_csv('./train_data_cleaning.csv')

In [12]:
X = data['text']
y = data['target']

In [13]:
words = []
data = []
labels = {}
labels_r = {}
idx = 0

In [14]:
for label in y:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [15]:
labels

{1: 1, 0: 2}

In [16]:
labels_r

{1: 1, 2: 0}

In [17]:
for Xb,yb in zip(X,y):
    Xb = tokenize(Xb)
    new_X = []
    for X_batch in Xb:
        new_X.append(stem(X_batch))
    words.extend(new_X)
    data.append([new_X,np.eye(labels[yb]+1,len(labels))[labels[yb]]])

In [18]:
words = sorted(set(words))

In [19]:
np.random.shuffle(data)

In [20]:
X = []
y = []

In [21]:
for sentence,tag in data:
    X.append(bag_of_words(sentence,words))
    y.append(tag)

In [22]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [23]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [24]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [27]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.iters = 10
        self.linear1 = Linear(len(words),512)
        self.linear2 = Linear(512,512)
        self.linear2bn = BatchNorm1d(512)
        self.linear3 = Linear(512,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.linear2bn(self.linear2(preds)))
        preds = self.linear3(preds)
        return preds

In [28]:
model = Model().to(device)
criteiron = MSELoss()

In [29]:
optimizer = Adam(model.parameters(),lr=0.001)

In [30]:
batch_size = 32
epochs = 100

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criteiron(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    torch.cuda.empty_cache()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()